In [1]:
from Services.DataSetProcessor import process_datasets
import fasttext
ft_model = fasttext.load_model('./Models/quora_model.bin')
process_datasets('quora',1,ft_model)

Processing documents with FastText model: 100%|██████████| 522931/522931 [04:30<00:00, 1931.93it/s]


Processed documents successfully saved to ./DataSets/Quora/embeddings.json


In [2]:
import ijson
from tqdm import tqdm
from Services.MilvusDB import MilvusDB
from Services.TextProcessor import TextProcessor

MilvusDB.connect()
MilvusDB.drop_all_collections()

filename = './DataSets/Quora/embeddings.json'
collection = MilvusDB.get_collection('qoura')
if collection:
    collection.drop()
if collection is None:
    collection = MilvusDB.create_collection('quora', 600)

batch_size = 1000  
batch = []

with open(filename, 'r', encoding='utf-8') as file:
    objects = ijson.items(file, 'item')
    for doc in tqdm(objects, desc="Inserting processed documents into MilvusDB"):
        entity = {
            "doc_id": doc["doc_id"],
            "vector": TextProcessor.normalize_vector(doc["vector"])
        }
        batch.append(entity)
        
        # Insert batch when it reaches the batch size
        if len(batch) >= batch_size:
            collection.insert(batch)
            batch = []  # Reset batch after insertion

    # Insert any remaining documents in the batch
    if batch:
        collection.insert(batch)

Connected to Milvus at localhost:19530
Error accessing collection: <SchemaNotReadyException: (code=1, message=Collection 'qoura' not exist, or you can pass in schema to create one.)>
Index created on vector with type IVF_FLAT and metric COSINE


Inserting processed documents into MilvusDB: 522931it [29:09, 298.94it/s]
